<a href="https://colab.research.google.com/github/VitaliyShein/Clusterizer/blob/main/Clusterizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%python -m pip install pip<24.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


Импорт textract

In [6]:
%pip install textract
import textract

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


Загрузка BERT

In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import torch

# Загрузка предобученной модели и токенизатора
model_name = 'bert-base-multilingual-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)
"""
    #Инициализация мультиязычных моделей
    # Классификатор
    tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
    classifier = BertForSequenceClassification.from_pretrained(
        "bert-base-multilingual-cased",
        num_labels=len(CLASS_NAMES)
    )
"""

# Универсальная модель для суммаризации
summarizer = pipeline(
    "summarization",
    model = model_name,
    tokenizer = tokenizer
    #model="facebook/mbart-large-50-many-to-many-mmt",  # Мультиязычная модель
    #tokenizer="facebook/mbart-large-50-many-to-many-mmt"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mo

In [8]:
def summarize_text(text, summarizer):
    """Генерация краткого описания"""
    try:
        # Для очень длинных текстов берем первые 1000 токенов
        summary = summarizer(
            text[:2000] if len(text) > 2000 else text,
            max_length=60,
            min_length=20,
            do_sample=False
        )
        return summary[0]['summary_text']
    except Exception as e:
        print(f"Summarization error: {e}")
        return "No summary available"

In [3]:
def DefineCluster(text, directory, filename):
  # Текст для классификации
  #text = "Article: reasearch the effect of garbage on the human psyche"

  # Токенизация и подготовка входа
  inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)

  # Классификация
  with torch.no_grad():
      outputs = model(**inputs)
      predictions = torch.argmax(outputs.logits, dim=1)

  # Соответствие меток
  labels = ["email", "scientific_article", "resume"]
  predicted_label = labels[predictions.item()]

  print(f"\nDocument classified as: {predicted_label}")

   # Вызов соответствующей функции обработки
  if predicted_label == "email":
    source_file = filename
    destination_path = "/content/drive/MyDrive/emails"

    # Создать папку, если её нет
    if not os.path.exists(destination_path):
        os.makedirs(destination_path)
    begPath = directory + '/' + filename
    shutil.copy(begPath, '/content/drive/MyDrive/emails')  #  os.rename(source_file, destination_path)
    print(f"Файл перемещён в {destination_path}")

  elif predicted_label == "scientific_article":
    source_file = filename
    destination_path = "/content/drive/MyDrive/scientific_article"

    # Создать папку, если её нет
    if not os.path.exists(destination_path):
        os.makedirs(destination_path)
    begPath = directory + '/' + filename
    shutil.copy(begPath, '/content/drive/MyDrive/scientific_article')  #  os.rename(source_file, destination_path)
    print(f"Файл перемещён в {destination_path}")

  elif predicted_label == "resume":
    source_file = filename
    destination_path = "/content/drive/MyDrive/resume"

    # Создать папку, если её нет
    if not os.path.exists(destination_path):
        os.makedirs(destination_path)
    begPath = directory + '/' + filename
    shutil.copy(begPath, '/content/drive/MyDrive/resume')  #  os.rename(source_file, destination_path)
    print(f"Файл перемещён в {destination_path}")
  else:
        print("Unknown document type")

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [9]:
import os
import shutil

# Путь к папке с файлами
folder_path = "/content/drive/MyDrive/InputData"

# Поддерживаемые форматы
supported_extensions = ['.pdf', '.docx', '.txt', '.pptx', '.jpg', '.png', '.jpeg']

# Проходим по всем файлам в папке
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    # Проверяем расширение файла
    if os.path.isfile(file_path) and any(file_path.lower().endswith(ext) for ext in supported_extensions):
        try:
          print(f"Обработка файла: {filename}...")

          # Извлекаем текст
          text = textract.process(file_path).decode('utf-8')

          print(summarize_text(text, summarizer))

          DefineCluster(text, folder_path, filename)

          summarize_text(text, summarizer)
        except ValueError:
          print("ERROR")


Обработка файла: em1.png...
Summarization error: 'SummarizationPipeline' object has no attribute 'prefix'
No summary available

Document classified as: resume
Файл перемещён в /content/drive/MyDrive/resume
Summarization error: 'SummarizationPipeline' object has no attribute 'prefix'
Обработка файла: em2.png...
Summarization error: 'SummarizationPipeline' object has no attribute 'prefix'
No summary available

Document classified as: resume
Файл перемещён в /content/drive/MyDrive/resume
Summarization error: 'SummarizationPipeline' object has no attribute 'prefix'
Обработка файла: em3.png...


KeyboardInterrupt: 

Определение кластера текста с помощью BERT